In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os,sys
from pathlib import Path
import json 
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-7s %(message)s',
                    stream=sys.stderr, level=logging.INFO)
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.INFO)

#General ML 
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, silhouette_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from itertools import combinations

#In-house Module Imports
from config import Configuration 
from datasets import EclipseSampledDataset, VoltaSampledDataset
from utils import *

In [4]:
user = "aksar"
logging.warning(f'Are you sure that you are: {user}?')

2022-02-19 16:35:13,572 WARNING Are you sure that you are: aksar?


In [5]:
#Update these if you are not the desired user
OUTPUT_DIR = f'/projectnb/peaclab-mon/{user}/unseen_experiments'
SYSTEM = 'volta'
EXP_NAME = 'tpds_experiments'
CV_INDEX = 0
FEATURE_SELECTION = False
SCALER = 'None' #For now, do the scaling inside the notebook, then you can move that to the class function
MODEL_CONFIG = 'experiment_1' #rf_tuncer or rf_tuncer_worst_case
logging.warning('Results will be generated in {}, double check please!'.format(MODEL_CONFIG))

2022-02-19 16:35:13,600 WARNING Results will be generated in experiment_1, double check please!


In [6]:
conf = Configuration(ipython=True,
                     overrides={
                         'output_dir': Path(OUTPUT_DIR), #change
                         'system' : SYSTEM,
                         'exp_name':EXP_NAME,                                                  
                         'cv_fold':CV_INDEX, 
                         'model_config': MODEL_CONFIG,
                     })

with open(str(conf['experiment_dir']) + '/anom_dict.json') as f:
    ANOM_DICT = json.load(f)
with open(str(conf['experiment_dir']) + '/app_dict.json') as f:
    APP_DICT = json.load(f)    
    
APP_REVERSE_DICT = {}
for app_name, app_encoding in APP_DICT.items():
    APP_REVERSE_DICT[app_encoding] = app_name    

ANOM_REVERSE_DICT = {}
for anom_name, anom_encoding in ANOM_DICT.items():
    ANOM_REVERSE_DICT[anom_encoding] = anom_name        

2022-02-19 16:35:13,630 WARNING You set windowing False, but you are trying to define window_size parameter, it is automatically set to 0. Please double check.
2022-02-19 16:35:13,631 INFO    Setting directory names
2022-02-19 16:35:13,637 INFO    Model config folder already exists, be careful, otherwise it will overwrite!
2022-02-19 16:35:13,641 INFO    Saving configuration as CSV


# The configuration used for this run:
# {'cv_fold': 0,
#  'exp_name': 'tpds_experiments',
#  'experiment_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments'),
#  'feature_extract': False,
#  'feature_select': False,
#  'hdf_data_path': PosixPath('/projectnb/peaclab-mon/aksar/datasets/tpds_data_hdfs'),
#  'metadata_path': None,
#  'model_config': 'experiment_1',
#  'model_config_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_1'),
#  'model_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_1/model'),
#  'num_split': 5,
#  'operation': 'read',
#  'output_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta'),
#  'plots_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_1/model/plots'),
#  'processed_ldms_data_path': None,
#  'raw_ldms_data_path': None,
#  'results_dir': Po

In [7]:
if SYSTEM == 'eclipse':
    
    eclipseDataset = EclipseSampledDataset(conf)
    train_data, train_label, test_data, test_label = eclipseDataset.load_dataset(scaler=SCALER,borghesi=False,tsfresh = True)

elif SYSTEM == 'volta':
    voltaDataset = VoltaSampledDataset(conf)
    train_data, train_label, test_data, test_label = voltaDataset.load_dataset(scaler=SCALER,borghesi=False)
    
assert list(train_data.index) == list(train_label.index) #check the order of the labels     
assert list(test_data.index) == list(test_label.index) #check the order of the labels    

if FEATURE_SELECTION:
    selected_features = pd.read_csv(conf['experiment_dir'] / 'selected_features.csv')
    train_data = train_data[list(selected_features['0'].values)]
    test_data = test_data[list(selected_features['0'].values)]
    
train_label['anom_names'] = train_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
train_label['app_names']=train_label['app'].apply(lambda x: APP_REVERSE_DICT[x])
test_label['anom_names'] = test_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
test_label['app_names']=test_label['app'].apply(lambda x: APP_REVERSE_DICT[x])

all_data = pd.concat([train_data, test_data])
all_data = all_data.dropna(axis=1, how='any')
all_label = pd.concat([train_label,test_label])

train_data = all_data.loc[train_label.index]
test_data = all_data.loc[test_label.index]
    
logging.info("Train data shape %s",train_data.shape)
logging.info("Train label shape %s",train_label.shape)
logging.info("Test data shape %s",test_data.shape)  
logging.info("Test label shape %s",test_label.shape)

logging.info("Train data label dist: \n%s",train_label['anom'].value_counts())
logging.info("Test data label dist: \n%s",test_label['anom'].value_counts())        

2022-02-19 16:35:13,681 INFO    BaseDataset Class Initialization
2022-02-19 16:35:13,682 INFO    HPCDataset Class Initialization
2022-02-19 16:35:13,682 INFO    VoltaSampledDataset Class Initialization
2022-02-19 16:35:13,804 INFO    Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-02-19 16:35:15,216 INFO    Train data shape (14640, 1441)
2022-02-19 16:35:15,218 INFO    Train label shape (14640, 2)
2022-02-19 16:35:15,218 INFO    Test data shape (6275, 1441)
2022-02-19 16:35:15,219 INFO    Test label shape (6275, 2)
2022-02-19 16:35:15,220 WARNING Beware that no scaling method is applied
2022-02-19 16:35:16,439 INFO    Train data shape (14640, 1441)
2022-02-19 16:35:16,440 INFO    Train label shape (14640, 4)
2022-02-19 16:35:16,441 INFO    Test data shape (6275, 1441)
2022-02-19 16:35:16,442 INFO    Test label shape (6275, 4)
2022-02-19 16:35:16,447 INFO    Train data label dist: 
0    13281
2      342
1      342
3      338
4      3

## Unknown App Experiments

In [8]:
SCALER = 'MinMax'

In [9]:
nas_apps = ['lu','sp','ft','bt','cg','mg']
mantevo_apps = ['miniMD','CoMD','miniGhost','miniAMR']
other_apps = ['kripke']

all_app_names = list(APP_DICT.keys())

assert set(nas_apps + mantevo_apps + other_apps) == set(all_app_names)


cluster_one = ['ft','cg','mg']
cluster_two = ['miniAMR','lu','miniGhost']
cluster_three = ['sp','bt','miniMD','kripke','CoMD']

In [10]:
all_test_app_groups = []
all_test_app_sets = []
for app_one in cluster_one:
    for app_two in cluster_two:
        for app_three in cluster_three:
            all_test_app_groups.append([app_one,app_two,app_three])
            all_test_app_sets.append([app_one,app_two,app_three])

In [11]:
#This doesn't hold the unique combinations, however it should have the total number
train_app_combinations = []

#45 different test_apps combination
for test_apps in all_test_app_groups:
    
    train_apps_all = list(set(all_app_names) - set(test_apps))
    #logging.info("Test apps: %s", test_apps)
    #logging.info("All Train apps: %s", train_apps_all)

    if len(train_apps_all) == 8:
        num_train_unknowns = [2,4,6,8]

    elif len(train_apps_all) == 5:
        num_train_unknowns = [3,5]

    else:
        raise

    #127 different train_apps combination
    for num_train_unknown in num_train_unknowns:
        for train_apps in combinations(train_apps_all, num_train_unknown):
            train_app_combinations.append(train_apps)

In [39]:
not_covered_combinations = [] #Not using this
unique_counter_dict = {}
extra_filenames = []
delete_extra_files = False

for file in listdir(conf['results_dir']):
    
    if "result_dict" in file:

        filename = file.split('.')[0]
        splitted_filename = file.split('#')    

        train_apps = set(splitted_filename[0].split(':')[1].split("-"))
        test_apps = set(splitted_filename[1].split(':')[1].split("-"))
        
        if not (frozenset(train_apps),frozenset(test_apps)) in unique_counter_dict:
            unique_counter_dict[(frozenset(train_apps),frozenset(test_apps))] = 1
        else:
            unique_counter_dict[(frozenset(train_apps),frozenset(test_apps))] += 1
            extra_filenames.append(file)
                    
        if (not test_apps in all_test_app_sets) and (not train_apps in train_app_combinations):
            not_covered_combinations.append([train_apps,test_apps])
            
sorted_unique_counter_dict = dict(sorted(unique_counter_dict.items(), key=lambda x: x[1], reverse=True))            

if delete_extra_files:
    
    for filename in extra_filenames:

        generic_filename = "#".join(filename.split("#")[:-1])

        os.remove(Path(conf['results_dir']) / filename)
        os.remove(Path(conf['results_dir']) / (generic_filename + "#alert_dict.json"))    

In [ ]:
counter = 0
for test_apps in all_test_app_groups:
    
    train_apps_all = list(set(all_app_names) - set(test_apps))
    logging.info("Test apps: %s", test_apps)
    logging.info("All Train apps: %s", train_apps_all)
    
    test_apps_label = all_label[all_label['app_names'].isin(test_apps)]
    assert set(test_apps_label['app_names'].unique()) == set(test_apps)
    
    test_apps_data = all_data.loc[test_apps_label.index]
    assert list(test_apps_data.index) == list(test_apps_label.index) #check the order of the labels       
    
    if len(train_apps_all) == 8:
        num_train_unknowns = [2,4,6,8]

    elif len(train_apps_all) == 5:
        num_train_unknowns = [3,5]
    
    else:
        raise
            
    for num_train_unknown in num_train_unknowns:    
        
        for train_apps_selected in combinations(train_apps_all, num_train_unknown):
            
            train_apps_selected = list(train_apps_selected)
            logging.info("Selected train apps: %s", train_apps_selected)                    
            
            if not (frozenset(train_apps_selected),frozenset(test_apps)) in unique_counter_dict:            
            
                train_apps_label = all_label[all_label['app_names'].isin(train_apps_selected)]
                train_apps_data = all_data.loc[train_apps_label.index]
                assert list(train_apps_data.index) == list(train_apps_label.index) #check the order of the labels 

                if SCALER == 'MinMax':

                    scaler = MinMaxScaler()    

                elif SCALER == 'Standard':

                    scaler = StandardScaler()
                    
                clf = RandomForestClassifier(random_state=0)
                
                pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])    
                
                fit_clf = pipeline.fit(train_apps_data, train_apps_label['anom'])

                pred_test = pipeline.predict(test_apps_data)
                logging.info("\n %s",classification_report(test_apps_label['anom'],pred_test, target_names=ANOM_DICT))    

                result_filename = "train:" + "-".join(train_apps_selected) + "#test:" + "-".join(test_apps) \
                                + "#model:" + clf.__class__.__name__ + "#cv:" + str(CV_INDEX)                
                result_dict = classification_report(test_apps_label['anom'],pred_test, target_names=ANOM_DICT,output_dict=True)

                with open(Path(conf['results_dir']) / (result_filename + "#result_dict.json"), "w") as fp:
                    json.dump(result_dict, fp)

                FAR_AMR_Calculate(true_label=test_apps_label['anom'],pred_label=pred_test,
                                  result_dir=conf['results_dir'],
                                  save_name=result_filename,save=True)      
                
            else: 
                counter += 1
                logging.info("File exists")

2022-02-19 00:13:50,373 INFO    Test apps: ['ft', 'miniAMR', 'sp']
2022-02-19 00:13:50,374 INFO    All Train apps: ['lu', 'cg', 'miniMD', 'bt', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,420 INFO    Selected train apps: ['lu', 'cg']
2022-02-19 00:13:50,420 INFO    File exists
2022-02-19 00:13:50,421 INFO    Selected train apps: ['lu', 'miniMD']
2022-02-19 00:13:50,422 INFO    File exists
2022-02-19 00:13:50,422 INFO    Selected train apps: ['lu', 'bt']
2022-02-19 00:13:50,423 INFO    File exists
2022-02-19 00:13:50,423 INFO    Selected train apps: ['lu', 'mg']
2022-02-19 00:13:50,424 INFO    File exists
2022-02-19 00:13:50,425 INFO    Selected train apps: ['lu', 'CoMD']
2022-02-19 00:13:50,425 INFO    File exists
2022-02-19 00:13:50,426 INFO    Selected train apps: ['lu', 'kripke']
2022-02-19 00:13:50,426 INFO    File exists
2022-02-19 00:13:50,427 INFO    Selected train apps: ['lu', 'miniGhost']
2022-02-19 00:13:50,427 INFO    File exists
2022-02-19 00:13:50,428 INFO    

2022-02-19 00:13:50,495 INFO    File exists
2022-02-19 00:13:50,495 INFO    Selected train apps: ['cg', 'miniMD', 'bt', 'miniGhost']
2022-02-19 00:13:50,496 INFO    File exists
2022-02-19 00:13:50,497 INFO    Selected train apps: ['cg', 'miniMD', 'mg', 'CoMD']
2022-02-19 00:13:50,497 INFO    File exists
2022-02-19 00:13:50,498 INFO    Selected train apps: ['cg', 'miniMD', 'mg', 'kripke']
2022-02-19 00:13:50,498 INFO    File exists
2022-02-19 00:13:50,499 INFO    Selected train apps: ['cg', 'miniMD', 'mg', 'miniGhost']
2022-02-19 00:13:50,499 INFO    File exists
2022-02-19 00:13:50,500 INFO    Selected train apps: ['cg', 'miniMD', 'CoMD', 'kripke']
2022-02-19 00:13:50,501 INFO    File exists
2022-02-19 00:13:50,501 INFO    Selected train apps: ['cg', 'miniMD', 'CoMD', 'miniGhost']
2022-02-19 00:13:50,502 INFO    File exists
2022-02-19 00:13:50,502 INFO    Selected train apps: ['cg', 'miniMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,503 INFO    File exists
2022-02-19 00:13:50,503 INFO

2022-02-19 00:13:50,563 INFO    Selected train apps: ['miniMD', 'bt', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,563 INFO    File exists
2022-02-19 00:13:50,564 INFO    Selected train apps: ['lu', 'cg', 'miniMD', 'bt', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,564 INFO    File exists
2022-02-19 00:13:50,565 INFO    Test apps: ['ft', 'miniAMR', 'bt']
2022-02-19 00:13:50,566 INFO    All Train apps: ['lu', 'cg', 'miniMD', 'sp', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,673 INFO    Selected train apps: ['lu', 'cg']
2022-02-19 00:13:50,674 INFO    File exists
2022-02-19 00:13:50,674 INFO    Selected train apps: ['lu', 'miniMD']
2022-02-19 00:13:50,675 INFO    File exists
2022-02-19 00:13:50,676 INFO    Selected train apps: ['lu', 'sp']
2022-02-19 00:13:50,676 INFO    File exists
2022-02-19 00:13:50,677 INFO    Selected train apps: ['lu', 'mg']
2022-02-19 00:13:50,677 INFO    File exists
2022-02-19 00:13:50,678 INFO    Selected train apps: ['lu', '

2022-02-19 00:13:50,747 INFO    Selected train apps: ['cg', 'miniMD', 'sp', 'mg']
2022-02-19 00:13:50,747 INFO    File exists
2022-02-19 00:13:50,748 INFO    Selected train apps: ['cg', 'miniMD', 'sp', 'CoMD']
2022-02-19 00:13:50,748 INFO    File exists
2022-02-19 00:13:50,749 INFO    Selected train apps: ['cg', 'miniMD', 'sp', 'kripke']
2022-02-19 00:13:50,750 INFO    File exists
2022-02-19 00:13:50,750 INFO    Selected train apps: ['cg', 'miniMD', 'sp', 'miniGhost']
2022-02-19 00:13:50,751 INFO    File exists
2022-02-19 00:13:50,751 INFO    Selected train apps: ['cg', 'miniMD', 'mg', 'CoMD']
2022-02-19 00:13:50,752 INFO    File exists
2022-02-19 00:13:50,752 INFO    Selected train apps: ['cg', 'miniMD', 'mg', 'kripke']
2022-02-19 00:13:50,753 INFO    File exists
2022-02-19 00:13:50,754 INFO    Selected train apps: ['cg', 'miniMD', 'mg', 'miniGhost']
2022-02-19 00:13:50,754 INFO    File exists
2022-02-19 00:13:50,755 INFO    Selected train apps: ['cg', 'miniMD', 'CoMD', 'kripke']
2022

2022-02-19 00:13:50,814 INFO    File exists
2022-02-19 00:13:50,815 INFO    Selected train apps: ['cg', 'miniMD', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,815 INFO    File exists
2022-02-19 00:13:50,816 INFO    Selected train apps: ['cg', 'sp', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,817 INFO    File exists
2022-02-19 00:13:50,817 INFO    Selected train apps: ['miniMD', 'sp', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,818 INFO    File exists
2022-02-19 00:13:50,818 INFO    Selected train apps: ['lu', 'cg', 'miniMD', 'sp', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,819 INFO    File exists
2022-02-19 00:13:50,819 INFO    Test apps: ['ft', 'miniAMR', 'miniMD']
2022-02-19 00:13:50,820 INFO    All Train apps: ['lu', 'cg', 'bt', 'sp', 'mg', 'CoMD', 'kripke', 'miniGhost']
2022-02-19 00:13:50,929 INFO    Selected train apps: ['lu', 'cg']
2022-02-19 00:13:50,930 INFO    File exists
2022-02-19 00:13:50,930 INFO    Selected train apps: 

...

2022-02-19 01:13:30,622 INFO    Total misclassified normal runs: 1759, Total normal runs 5265 
2022-02-19 01:13:30,622 INFO    FAR: 0.33409306742640077
2022-02-19 01:13:30,623 INFO    Total misclassified anom runs: 78, Total anom runs 540 
2022-02-19 01:13:30,624 INFO    AMR: 0.14444444444444443
2022-02-19 01:13:30,626 INFO    Selected train apps: ['bt', 'kripke']
2022-02-19 01:13:38,315 INFO    
               precision    recall  f1-score   support

        none       0.98      0.99      0.98      5265
       dcopy       1.00      0.98      0.99       135
    memeater       1.00      1.00      1.00       135
        leak       1.00      1.00      1.00       135
        dial       0.28      0.19      0.23       135

    accuracy                           0.97      5805
   macro avg       0.85      0.83      0.84      5805
weighted avg       0.96      0.97      0.97      5805

2022-02-19 01:13:38,328 INFO    Total misclassified normal runs: 65, Total normal runs 5265 
2022-02-19 01:13:

2022-02-19 01:15:11,545 INFO    AMR: 0.14814814814814814
2022-02-19 01:15:11,547 INFO    Selected train apps: ['lu', 'miniAMR', 'miniMD', 'CoMD']
2022-02-19 01:15:20,716 INFO    
               precision    recall  f1-score   support

        none       0.98      1.00      0.99      5265
       dcopy       1.00      0.63      0.77       135
    memeater       1.00      1.00      1.00       135
        leak       1.00      1.00      1.00       135
        dial       0.85      0.33      0.48       135

    accuracy                           0.98      5805
   macro avg       0.96      0.79      0.85      5805
weighted avg       0.97      0.98      0.97      5805

2022-02-19 01:15:20,729 INFO    Total misclassified normal runs: 0, Total normal runs 5265 
2022-02-19 01:15:20,730 INFO    FAR: 0.0
2022-02-19 01:15:20,731 INFO    Total misclassified anom runs: 132, Total anom runs 540 
2022-02-19 01:15:20,732 INFO    AMR: 0.24444444444444444
2022-02-19 01:15:20,733 INFO    Selected train apps:

2022-02-19 01:17:11,634 INFO    
               precision    recall  f1-score   support

        none       0.99      1.00      0.99      5265
       dcopy       0.98      0.90      0.94       135
    memeater       1.00      1.00      1.00       135
        leak       1.00      1.00      1.00       135
        dial       0.81      0.41      0.54       135

    accuracy                           0.98      5805
   macro avg       0.96      0.86      0.90      5805
weighted avg       0.98      0.98      0.98      5805

2022-02-19 01:17:11,647 INFO    Total misclassified normal runs: 2, Total normal runs 5265 
2022-02-19 01:17:11,648 INFO    FAR: 0.0003798670465337132
2022-02-19 01:17:11,649 INFO    Total misclassified anom runs: 80, Total anom runs 540 
2022-02-19 01:17:11,649 INFO    AMR: 0.14814814814814814
2022-02-19 01:17:11,651 INFO    Selected train apps: ['lu', 'cg', 'miniMD', 'CoMD']
2022-02-19 01:17:22,595 INFO    
               precision    recall  f1-score   support

        

2022-02-19 01:19:18,602 INFO    Total misclassified normal runs: 2, Total normal runs 5265 
2022-02-19 01:19:18,602 INFO    FAR: 0.0003798670465337132
2022-02-19 01:19:18,603 INFO    Total misclassified anom runs: 83, Total anom runs 540 
2022-02-19 01:19:18,604 INFO    AMR: 0.1537037037037037
2022-02-19 01:19:18,605 INFO    Selected train apps: ['lu', 'miniMD', 'ft', 'CoMD']
2022-02-19 01:19:30,096 INFO    
               precision    recall  f1-score   support

        none       0.98      1.00      0.99      5265
       dcopy       1.00      0.96      0.98       135
    memeater       1.00      1.00      1.00       135
        leak       1.00      1.00      1.00       135
        dial       0.88      0.33      0.48       135

    accuracy                           0.98      5805
   macro avg       0.97      0.86      0.89      5805
weighted avg       0.98      0.98      0.98      5805

2022-02-19 01:19:30,128 INFO    Total misclassified normal runs: 0, Total normal runs 5265 
2022-0

2022-02-19 01:21:13,756 INFO    FAR: 0.0
2022-02-19 01:21:13,757 INFO    Total misclassified anom runs: 85, Total anom runs 540 
2022-02-19 01:21:13,758 INFO    AMR: 0.1574074074074074
2022-02-19 01:21:13,760 INFO    Selected train apps: ['miniAMR', 'cg', 'miniMD', 'ft']
2022-02-19 01:21:27,383 INFO    
               precision    recall  f1-score   support

        none       0.99      1.00      0.99      5265
       dcopy       0.98      0.97      0.98       135
    memeater       1.00      1.00      1.00       135
        leak       1.00      1.00      1.00       135
        dial       0.85      0.65      0.74       135

    accuracy                           0.99      5805
   macro avg       0.96      0.92      0.94      5805
weighted avg       0.99      0.99      0.99      5805

2022-02-19 01:21:27,420 INFO    Total misclassified normal runs: 14, Total normal runs 5265 
2022-02-19 01:21:27,420 INFO    FAR: 0.0026590693257359924
2022-02-19 01:21:27,422 INFO    Total misclassified a

### 5-fold CV Validation

In [29]:
SCALER = 'MinMax'

if SCALER == 'MinMax':
    
    scaler = MinMaxScaler()    

elif SCALER == 'Standard':
    
    scaler = StandardScaler()

clf = RandomForestClassifier(random_state=0)

pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])

In [31]:
scoring = ['precision_macro', 'recall_macro', 'f1_macro', 'f1_weighted']
skf = StratifiedKFold(n_splits=5)

scores = cross_validate(pipeline, all_data, all_label['anom'].values,                         
                        cv=skf, 
                        scoring=scoring)

In [32]:
scores

{'fit_time': array([37.87093496, 33.30407429, 31.670362  , 32.61878467, 33.78410292]),
 'score_time': array([0.22262263, 0.16374779, 0.16079235, 0.16500545, 0.16272569]),
 'test_precision_macro': array([0.99701681, 0.9971924 , 0.99916055, 0.99936975, 0.99889966]),
 'test_recall_macro': array([0.97295918, 0.96869731, 0.96666667, 0.975     , 0.95625   ]),
 'test_f1_macro': array([0.98429409, 0.9816149 , 0.98139757, 0.98635104, 0.97488691]),
 'test_f1_weighted': array([0.9967994 , 0.99602717, 0.99600515, 0.99703788, 0.99467834])}